In [14]:
# Import lại toàn bộ
import pandas as pd
import numpy as np
import random
from faker import Faker
import hashlib

In [15]:
fake = Faker('en_US')
np.random.seed(42)
random.seed(42)

In [ ]:
provinces = [
    'Hà Nội', 'Hồ Chí Minh', 'Hải Phòng', 'Đà Nẵng', 'Cần Thơ',
    'An Giang', 'Bà Rịa - Vũng Tàu', 'Bắc Giang', 'Bắc Kạn', 'Bạc Liêu',
    'Bắc Ninh', 'Bến Tre', 'Bình Định', 'Bình Dương', 'Bình Phước',
    'Bình Thuận', 'Cà Mau', 'Cao Bằng', 'Đắk Lắk', 'Đắk Nông',
    'Điện Biên', 'Đồng Nai', 'Đồng Tháp', 'Gia Lai', 'Hà Giang',
    'Hà Nam', 'Hà Tĩnh', 'Hải Dương', 'Hậu Giang', 'Hòa Bình',
    'Hưng Yên', 'Khánh Hòa', 'Kiên Giang', 'Kon Tum', 'Lai Châu',
    'Lâm Đồng', 'Lạng Sơn', 'Lào Cai', 'Long An', 'Nam Định',
    'Nghệ An', 'Ninh Bình', 'Ninh Thuận', 'Phú Thọ', 'Phú Yên',
    'Quảng Bình', 'Quảng Nam', 'Quảng Ngãi', 'Quảng Ninh', 'Quảng Trị',
    'Sóc Trăng', 'Sơn La', 'Tây Ninh', 'Thái Bình', 'Thái Nguyên',
    'Thanh Hóa', 'Thừa Thiên Huế', 'Tiền Giang', 'Trà Vinh', 'Tuyên Quang',
    'Vĩnh Long', 'Vĩnh Phúc', 'Yên Bái'
]

# Channels
channels = ['GT', 'MT', 'Online']

# SKU & Tỷ trọng
skus = {
    'Pro-Boost 110ml': {'price': 7200, 'cost': 5000, 'weight': 5},
    'Pro-Boost 180ml': {'price': 10200, 'cost': 7000, 'weight': 5},
    'Pro-Boost 220ml': {'price': 12000, 'cost': 8500, 'weight': 3},
    'Pro-Boost Kids 110ml': {'price': 6900, 'cost': 4800, 'weight': 1},
    'Pro-Boost Less Sugar 180ml': {'price': 9800, 'cost': 7200, 'weight': 1},
}
# Tạo viết tắt tỉnh
def abbreviate_province(province):
    return ''.join(word[0].upper() for word in province.split() if word[0].isalpha())

In [17]:
# Thời gian
start_date = pd.to_datetime("2025-01-01")
end_date = pd.to_datetime("2025-06-30")
dates = pd.date_range(start=start_date, end=end_date)

In [20]:
# Sinh danh sách khách hàng cố định
customer_ids = [f"CUST{str(i).zfill(5)}" for i in range(random.randint(300, 2000))]

In [21]:
# Sinh dữ liệu
rows = []

for _ in range(70000):
    order_date = random.choice(dates)
    province = random.choices(provinces, weights=[3 if p in ['Hà Nội', 'Hồ Chí Minh'] else 1 for p in provinces])[0]
    channel = random.choice(channels)

    sku_names = list(skus.keys())
    weights = [skus[sku]['weight'] for sku in sku_names]
    sku = random.choices(sku_names, weights=weights, k=1)[0]

    unit_price = skus[sku]['price']
    cost = skus[sku]['cost']
    qty = np.random.randint(50, 500)

    revenue = qty * unit_price
    cogs = qty * cost
    profit = revenue - cogs

    lead_time = np.random.randint(1, 6)
    shipping_date = order_date + pd.Timedelta(days=lead_time)
    otif = 1 if lead_time <= 3 else 0

    stock_before = np.random.randint(qty + 100, qty + 1000)
    stock_after = stock_before - qty
    returned = int(qty * np.random.choice([0, 0.01, 0.02, 0.03], p=[0.85, 0.1, 0.04, 0.01]))
    promo_flag = random.choice([0, 1]) if order_date.month in [3, 4] else 0

    # Tạo Order_ID có tính khách hàng lặp lại
    customer_id = random.choice(customer_ids)
    prov_abbr = abbreviate_province(province)
    hash_id = hashlib.md5((str(order_date) + sku).encode()).hexdigest()[:5].upper()
    order_id = f"{prov_abbr}-{customer_id}-{hash_id}"

    rows.append([
        order_id, order_date.date(), province, channel, sku,
        unit_price, qty, revenue, cost, cogs, profit, lead_time,
        shipping_date.date(), otif, stock_before, stock_after, returned, promo_flag
    ])

# Tạo DataFrame
columns = [
    "Order_ID", "Order_Date", "Province", "Channel", "SKU",
    "Unit_Price", "Quantity", "Revenue", "Cost_Per_Unit", "COGS", "Gross_Profit",
    "Lead_Time_Days", "Shipping_Date", "OTIF_Flag",
    "Stock_Before", "Stock_After", "Returned_Units", "Promotion_Flag"
]
df = pd.DataFrame(rows, columns=columns)

In [22]:
# Lưu file
output_path = "D:/Dự Án mới/vinamilk_proboost_6months_final.csv"
df.to_csv(output_path, index=False, encoding='utf-8-sig')

print("✅ File đã tạo:", output_path)

✅ File đã tạo: D:/Dự Án mới/vinamilk_proboost_6months_final.csv
